In [1]:
import copy
import csv
import re
import json
import pandas as pd

#### 1) ライブラリ

##### パスの先頭のキーを取得する

In [2]:
def get_root_key(s_path):
    return s_path.split('/')[0]

##### パスの末尾のキーを取得する

In [3]:
def get_leaf_key(s_path):    
    return s_path.split('/')[-1]

##### パスの先頭のインデックスを取得する

In [6]:
def get_root_index(s_path):
    return int(re.findall(r'\[([0-9]+)\]',s_path.split('/')[0])[0])

##### パスの末尾のインデックスを取得する

In [7]:
def get_leaf_index(s_path):
    return int(re.findall(r'\[([0-9]+)\]',s_path.split('/')[-1])[0])

#### パスの先頭のキー・インデックスを除外する

In [4]:
def exclude_root_node(s_path):
    return '/'.join(s_path.split('/')[1:])

##### パスの末尾のキー・インデックスを除外する

In [5]:
def exclude_leaf_node(s_path):
    return '/'.join(s_path.split('/')[:-1])

##### keys_and_settings用のパスをcase用のパスに変更する

In [8]:
def get_path_for_case(s_path):
    return s_path.replace('/dictionary', '').replace('/list', '')

##### パスの最後にキーを追加して新しいパスを作成する

In [9]:
def make_new_path(s_path, key, addition):
    
    # パスの先頭のキーとなる場合
    if s_path == '':
        return key + addition
        
    # パスの先頭のキーとならない場合
    else:
        return s_path + '/' + key + addition

##### 指定されたオブジェクトを取得する

対応するオブジェクトが見つからない場合にはNoneを返す

In [10]:
def get_object_or_none_by_path(obj, s_path):
    
    # オブジェクトまたはパスがNoneの場合はNoneを返す
    if obj == None or s_path == None:
        return None

    # パスの指定がない場合は辞書をそのまま返す
    elif s_path == '' :
        return obj
    
    # キーが指定された場合はリストの要素または辞書の値を返す
    elif s_path.count('/') == 0: 
        
        # リストの場合
        if type(obj) == list:
            root_index = get_root_index(s_path)
            if root_index < len(obj):
                return obj[root_index]
            else:
                return None
                
        # 辞書の場合
        else:
            if s_path in obj:
                return obj[s_path]
            else:
                return None
    
    # パスが指定された場合は再帰呼び出し
    else:
        
        # リストの場合
        if type(obj) == list:
            root_index = get_root_index(s_path)
            if root_index < len(obj):
                return get_object_or_none_by_path(obj[root_index], exclude_root_node(s_path))
            else:
                return None
                
        # 辞書の場合
        else:
            if get_root_key(s_path) in obj:
                return get_object_or_none_by_path(obj[get_root_key(s_path)], exclude_root_node(s_path))
            else:
                return None

対応するオブジェクトが見つからない場合には例外処理

In [11]:
def get_object_or_exception_by_path(obj, s_path):
    
    obj_temp = get_object_or_none_by_path(obj, s_path)
    
    if obj_temp == None:
        raise Exception
    else:
        return obj_temp

##### 指定されたパスの辞書の指定されたキーの値を取得する

対応する値が見つからない場合にはNoneを返す

In [12]:
def get_value_or_none_by_key(d, key, s_path):
    
    # 辞書がNoneの場合はNoneを返す
    if d == None:
        return None
    
    # パスの指定がない場合は辞書をそのまま返す
    elif s_path == '' :
        
        # リストの場合
        if type(d) == list:
            root_index = get_root_index(key)
            if root_index < len(d):
                return d[root_index]
            else:
                return None
            
        # 辞書の場合
        else:          
            if key in d:
                return d[key]
            else:
                return None
    
    # キーが指定された場合はリストまたは辞書を返す
    elif s_path.count('/') == 0:    
        
        # リストの場合
        if type(d) == list:
            root_index = get_root_index(s_path)
            if root_index < len(d):
                return d[root_index][key]
            else:
                return None
            
        # 辞書の場合
        else:
            if s_path in d:
                if d[s_path] == None:
                    return None
                else:
                    if key in d[s_path]:
                        return d[s_path][key]
                    else:
                        return None
                    
    # パスが指定された場合は再帰呼び出し
    else:
        
        # リストの場合
        if type(d) == list:
            root_index = get_root_index(s_path)
            if root_index < len(d):
                return get_value_or_none_by_key(d[root_index], key, exclude_root_node(s_path))
            else:
                return None
            
        # 辞書の場合
        else:
            if get_root_key(s_path) in d:
                return get_value_or_none_by_key(d[get_root_key(s_path)], key, exclude_root_node(s_path))
            else:
                return None

対応する値が見つからない場合には例外処理

In [13]:
def get_value_or_exception_by_key(d, key, s_path):
    
    value_temp = get_value_or_none_by_key(d, key, s_path)
    
    if value_temp == None:
        raise Exception
    else:
        return value_temp

##### キーのパスを取得する

対応するキーのパスが見つからない場合にはNoneを返す

In [14]:
def find_key_path_or_none(d, key, s_path):
    
    d_target = get_object_or_none_by_path(d, s_path)
    
    # 対応する辞書が見つからない場合はNoneを返す
    if d_target == None:
        return None
    
    # 辞書にキーが含まれる場合はキーのパスを返す
    elif key in get_object_or_exception_by_path(d, s_path):
        return s_path + '/' + key
    
    # 辞書にキーが含まれない場合
    else:
        
        # 上位の辞書がある場合は再帰呼び出し
        if s_path.count('/') > 0:
            return find_key_path_or_none(d, key, exclude_leaf_node(s_path))
        
        # 上位の辞書がない場合はNoneを返す
        else:
            raise None

対応するキーのパスが見つからない場合には例外処理

In [15]:
def find_key_path_or_exception(d, key, s_path):
    
    path_temp = find_key_path_or_none(d, key, s_path)
    
    if path_temp == None:
        raise Exception
    else:
        return path_temp

##### キーの値を設定する

In [16]:
def set_value_by_key(d, s_path, value):
    
    # 辞書の取得
    d_target = get_object_or_exception_by_path(d, exclude_leaf_node(s_path))
    
    # 対応する辞書が見つからない場合にはNoneを返す
    if d_target == None:
        raise Exception
    
    # キーの値を設定する
    else:
        d_target[get_leaf_key(s_path)] = value

#### 2) 入力条件に該当しない項目をNoneとする

##### キーの値をNoneに設定する

In [17]:
def set_element_none(case, s_path):
    
    # 辞書の取得
    d_target = get_object_or_exception_by_path(case, exclude_leaf_node(s_path))    
    
    # Noneに設定
    if s_path == '' :
        case = None
    else:
        if type(case) == list:
            d_target[get_leaf_index(s_path)] = None
        else:
            d_target[get_leaf_key(s_path)] = None

##### 条件を満たさない項目をNoneに設定する

In [18]:
def set_unconditional_elements_none(keys_and_settings, case, s_path):
    
    # 辞書の取得
    d_target = get_object_or_exception_by_path(keys_and_settings, s_path)
    
    for key in d_target :
        
        # conditionsがある場合
        if 'conditions' in d_target[key]:
            set_typical = False
            # conditionsの中に全ての項目が条件を満たす組合せがある場合には、typicalを設定に該当
            for condition in d_target[key]['conditions']:                
                s_path_for_case = get_path_for_case(s_path)
                if all(get_object_or_none_by_path(case, find_key_path_or_exception(case, condition_key, s_path_for_case)) == condition[condition_key] for condition_key in condition): 
                    set_typical = True  
        else:
            set_typical = True
            
        # typicalを設定に該当しない場合には、Noneとする
        if set_typical == False:
            set_element_none(case, get_path_for_case(make_new_path(s_path, key, '')))
        
        # typicalを設定に該当し、下位項目がdictionaryの場合
        elif 'dictionary' in d_target[key]:
            set_unconditional_elements_none(keys_and_settings, case, make_new_path(s_path, key, '/dictionary'))
        
        # typicalを設定に該当し、下位項目がlistの場合
        elif 'list' in d_target[key]:
            set_unconditional_elements_none(keys_and_settings, case, make_new_path(s_path, key, '/list/[0]'))

#### 3) 基準ケース作成

##### 各項目をデフォルト値に設定する

In [19]:
def make_standard_case(keys_and_settings, s_path):
    
    standard_case = {}
    
    # 辞書の取得
    d_target = get_object_or_exception_by_path(keys_and_settings, s_path)
    
    # 基準ケースの作成
    for key in d_target:
        
        # 下位項目に辞書がある場合は再帰呼び出し
        if 'dictionary' in d_target[key]:
            standard_case[key] = make_standard_case(keys_and_settings, make_new_path(s_path, key, '/dictionary'))
        
        # 下位項目にリストがある場合は再帰呼び出し
        elif 'list' in d_target[key]:
            standard_case[key] = [make_standard_case(keys_and_settings, make_new_path(s_path, key, '/list/[0]'))]
        
        # 値入力の項目の設定
        elif d_target[key]['input'] == 'option' or d_target[key]['input'] == 'value':
            standard_case[key] = d_target[key]['typical']
            
        # 要素選択の項目の設定
        elif d_target[key]['input'] == 'text':
            standard_case[key] = d_target[key]['templete']
                    
    return standard_case

#### 4) 設定値リスト作成

##### リスト要素の追加・削除

In [20]:
def get_virtual_options(l_level, setting):
    
    l_temp = copy.deepcopy(l_level)
    
    # リスト要素の追加
    if 'extra' in setting:
        for element in setting['extra']:
            if element not in l_temp:
                l_temp.append(element)
    
    # リスト要素の削除
    if 'exception' in setting:
        for element in setting['exception']:
            if element in l_temp:
                l_temp.remove(element)
    
    return sorted(l_temp)

##### 設定値リスト作成

In [21]:
def make_level_list(setting):

    # ケース設定用のリスト作成
    if setting['input'] == 'option':
        temp = get_virtual_options(setting['options'], setting)    
        temp = [0] + temp + [max(temp) + 1]    

    # ケース設定用のリスト作成
    elif setting['input'] == 'value':
        temp = [
            setting['min'] - setting['dec'],
            setting['min'],
            setting['typical'],
            setting['max'],
            setting['max'] + setting['dec']
        ]
    
    return get_virtual_options(temp, setting)

#### 5) 項目の入力条件を満たすよう設定

##### 項目の入力条件を満たすよう上位項目のconditionに規定された項目の値を設定

In [22]:
def set_upper_keys(keys_and_settings, case, s_path):

    # 辞書の取得
    d_target = get_object_or_exception_by_path(keys_and_settings, s_path)

    # 上位の項目の条件を満たすよう設定する
    if type(d_target) == list:
        pass    
    elif 'conditions' in d_target:
        case = meet_condition(keys_and_settings, case, s_path) 
        
    # 上位の項目がある場合は再帰呼び出し
    if s_path.count('/') > 0: 
        set_upper_keys(keys_and_settings, case, exclude_leaf_node(s_path))

##### 項目の入力条件を満たすようconditionに規定された項目の値を設定

In [23]:
def meet_condition(keys_and_settings, case, s_path):
    
    # 辞書の取得
    d_target = get_object_or_exception_by_path(keys_and_settings, s_path)

    # conditionsを含む場合
    if 'conditions' in d_target:
    
        # リストの1番目の条件を満たすよう、リスト中の項目を設定する
        for condition_key in  d_target['conditions'][0]:

            value = d_target['conditions'][0][condition_key]  
            new_s_path = find_key_path_or_exception(keys_and_settings, condition_key, s_path)
            set_value_by_key(case, get_path_for_case(new_s_path), value)
            
            # リスト中の項目が入力条件を満たすよう設定する
            meet_condition(keys_and_settings, case, new_s_path)

    # 上位項目を有する場合
    if s_path.count('/') > 0:
        set_upper_keys(keys_and_settings, case, exclude_leaf_node(s_path))
            
    return case

#### 6) ケース作成

##### 項目の値を設定

In [24]:
def set_element_to_case(case, s_path, element):
    
    target_d = get_object_or_exception_by_path(case, exclude_leaf_key(s_path))
    target_d[get_leaf_key(s_path)] = element

##### 項目の値をリストの値に設定

In [25]:
def make_case(keys_and_settings, cases, standard_case, l_level, s_path):
    
        for element in l_level:
            
            # 基準ケースの複製
            case = copy.deepcopy(standard_case)    
            
            # 項目の値をリストの値に設定
            set_element_to_case(case, get_path_for_case(s_path), element)
            
            # 項目の入力条件を満たすよう設定
            meet_condition(keys_and_settings, case, s_path)
            
            # 入力条件に該当しない項目をNoneと設定
            set_unconditional_elements_none(keys_and_settings, case, '')   
            
            # 対象項目の追加
            case['target_key'] =  get_leaf_key(s_path) 
            
            # ケース設定
            cases.append(case)

##### 各項目の値をリストの値に設定

In [26]:
def make_cases(cases, standard_case, keys_and_settings, s_path):
    
    d_target = get_object_or_exception_by_path(keys_and_settings, s_path)
    
    # ケースの生成
    for key in d_target:
        
        #下位項目にdictionaryを有する場合
        if 'dictionary' in d_target[key] :
            new_s_path = make_new_path(s_path, key, '/dictionary')
            cases = make_cases(cases, standard_case, keys_and_settings, new_s_path)        
        
        #下位項目にlistを有する場合
        elif 'list' in d_target[key] :
            new_s_path = make_new_path(s_path, key, '/list/[0]')
            cases = make_cases(cases, standard_case, keys_and_settings, new_s_path)

        #要素選択・値入力の項目ケース作成
        elif d_target[key]['input'] == 'option' or d_target[key]['input'] == 'value' :
            new_s_path = make_new_path(s_path, key, '')
            l_level = make_level_list(d_target[key])
            make_case(keys_and_settings, cases, standard_case, l_level, new_s_path)

    return cases

#### 7) リストの項目の複製

##### リスト数の取得 ただし条件に適合する最小のリスト数の取得となっている

In [27]:
def get_n_list(keys_and_settings, case, s_path):
    
    # 辞書の取得
    d_target = get_object_or_exception_by_path(keys_and_settings, s_path)
    
    # リスト数が数で与えられている場合はその数を返す
    if type(d_target) == int:
        return d_target
    
    # リスト数が辞書で与えられている場合は条件に適合する最小のリスト数を返す
    else:
        s_path_for_case = get_path_for_case(exclude_leaf_key(s_path))
        for key in d_target:
            for condition in d_target[key]:
                if all(get_object_or_none_by_path(case, find_key_path_or_none(case, condition_key, s_path_for_case)) == condition[condition_key] for condition_key in condition):
                    return key
                
        # 該当する条件のない場合は複製しない（リスト数1）
        return 1

##### リスト複製

In [28]:
def copy_list(case, n_list, s_path):

    if n_list > 0 :
        
        # 辞書の取得
        d_target = get_object_or_none_by_path(case, s_path)
        
        # リスト複製
        if d_target != None:
            for i in range(0, n_list):
                d_target.append(copy.deepcopy(d_target[0]))

            for i in range(0, n_list):
                if 'name' in d_target[i]:
                    d_target[i]['name'] = d_target[i]['name'] + str(i + 1)

    return case

##### リストの項目の複製

In [29]:
def copy_case_list(keys_and_settings, case, s_path):
    
    # 辞書の取得
    d_target = get_object_or_exception_by_path(keys_and_settings, s_path)

    # リストの場合は下位項目について再帰呼び出し
    if type(d_target) == list:
        new_s_path = make_new_path(s_path, '[0]', '')
        return copy_case_list(keys_and_settings, case, new_s_path)
    
    else:
        for key in d_target:
            
            # リストを含む場合はリストを複製し下位項目について再帰呼び出し
            if 'list' in d_target[key]:
                new_s_path = make_new_path(s_path, key, '/list')
                case = copy_case_list(keys_and_settings, case, new_s_path)
                    
                n_list = get_n_list(keys_and_settings, case, s_path + '/' + key + '/n_list')
                case = copy_list(case, n_list, get_path_for_case(new_s_path))
            
            # 辞書を含む場合は下位項目について再帰呼び出し
            elif 'dictionary'in d_target[key]:                
                new_s_path = make_new_path(s_path, key, '/dictionary')
                case =  copy_case_list(keys_and_settings, case, new_s_path)
            
    return case

##### リストをコピーしてケースを上書き

In [30]:
def copy_cases_list(keys_and_settings, cases):

    for i, case in enumerate(cases):
        new_case = copy.deepcopy(case)
        cases[i] = copy_case_list(keys_and_settings, new_case, s_path='')
            
    return cases

#### 8) csv出力用リストの作成

##### 最大のリスト数の取得

In [31]:
def get_max_n_list(keys_and_settings, s_path):
    
    d_target = get_object_or_exception_by_path(keys_and_settings, s_path)
    
    # リスト数が数で与えられている場合はその数を返す
    if type(d_target['n_list']) == int:
        return d_target['n_list']
    
    # リスト数が辞書で与えられている場合は最大の数を返す
    else:
        return max(d_target['n_list'].keys())

##### csv出力用キーの作成

In [32]:
def make_keys(keys_and_settings, keys, s_path, s_path_for_d):
    
    # 辞書の取得
    d_target = get_object_or_exception_by_path(keys_and_settings, s_path_for_d)
    
    # リストの場合
    if type(d_target) == list:
        
        # 最大のリスト数の取得
        n_list = get_max_n_list(keys_and_settings, exclude_leaf_key(s_path_for_d))
        
        # 最大のリスト数のキーを作成
        for i in range(0, n_list):
            new_s_path = make_new_path(s_path, '[' + str(i) + ']', '')
            new_s_path_for_d = make_new_path(s_path_for_d , '[0]', '')
            keys = make_keys(keys_and_settings, keys, new_s_path, new_s_path_for_d)
            
    # 辞書の場合
    else:
        for key in d_target:
            if key != 'conditions' and  key != 'n_list':
                
                # 下位項目がある場合
                if 'input' not in d_target[key]: 
                    keys = make_keys(keys_and_settings, keys, make_new_path(s_path, key, ''), make_new_path(s_path_for_d, key, ''))

                # キーの追加
                else:
                    keys = keys + [get_path_for_case(make_new_path(s_path, key, ''))]
        
    return keys

##### csv出力用リストの作成

In [33]:
def make_cases_for_csv(cases, keys):
    
    cases_for_csv = []
    
    for case in cases:
        case_for_csv = {}
        for key in keys:
            case_for_csv[key] = get_value_or_none_by_key(case, get_leaf_key(key), exclude_leaf_key(key))
        cases_for_csv.append(case_for_csv) 
    
    return cases_for_csv

#### 9) 作成済ケースの確認

In [34]:
def check_existence(cases):
    
    existing_cases = {}
    
    for i, case in enumerate(cases):
    
        # 作成済ケースに一致するケースがある場合は、一致するケースの番号を取得
        t_case = tuple(case[key] for key in case if key != 'target_key')
        if t_case in existing_cases:
            case['existing_case'] = existing_cases[t_case] + 1

        # 作成済ケースに一致するケースがない場合は、作成済ケースに追加
        else:
            existing_cases[t_case] = i

#### 10) テストケース出力

In [35]:
def output_cases(keys_and_settings):

    # 基準ケースの作成
    standard_case = make_standard_case(keys_and_settings, s_path='')

    # テストケースの作成
    cases = make_cases([], standard_case, keys_and_settings, s_path='')    
    
    # リスト複製
    cases = copy_cases_list(keys_and_settings, cases)
    
    # csv出力用テストケースの作成
    keys = ['target_key'] + ['existing_case'] + make_keys(keys_and_settings, keys=[], s_path='', s_path_for_d='')
    cases_for_csv = make_cases_for_csv(cases, keys)
    
    # 重複確認
    check_existence(cases_for_csv)   

    # JSON出力
    for i, case in enumerate(cases):
        s_file_name = ('{:0=5}'.format(i + 1)) +'.json'
        with open('test//' + s_file_name, 'w') as f_case:
            json.dump(case, f_case, indent=4), 

    # csv出力
    with open("test_lv2_cases.csv", "w", encoding="Shift_jis") as csv_test_lv2_cases:
        csv_test_lv2_cases.write(','.join(keys) + '\n')

        for case in cases_for_csv:
            row = [(str(case[key]) if key in case else '') for key in keys]
            csv_test_lv2_cases.write(','.join(row) + '\n')

#### 11) 項目設定

In [36]:
keys_and_settings = {
'common': {'dictionary': {					
	'region': { 'input': 'option', 'options': [1, 2, 3, 4, 5, 6, 7, 8], 'typical': 1},				
	'main_occupant_room_floor_area': { 'input': 'value', 'dec': 0.01, 'min': 0, 'max': 999.99, 'typical': 29.81},				
	'other_occupant_room_floor_area': { 'input': 'value', 'dec': 0.01, 'min': 0, 'max': 999.99, 'typical': 51.34},				
	'total_floor_area': { 'input': 'value', 'dec': 0.01, 'min': 0, 'max': 999.99, 'typical': 120.08}				
}},					
'envelope': {'dictionary': {					
	'input_method': { 'input': 'option', 'options': [1, 2, 3, 4], 'exception': [1], 'typical': 1},				
	'simple_method': {'conditions': [{'input_method': 2}], 'dictionary': {				
		'insulation_type': { 'input': 'option', 'options': [1, 2], 'typical': 1},			
		'insulation_type_bathroom': { 'input': 'option', 'options': [1, 2, 3], 'typical': 1, 'conditions': [{'insulation_type': 1}]},			
		'u_value_roof': { 'input': 'value', 'dec': 0.01, 'min': 0, 'max': 10, 'typical': 0.24},			
		'u_value_wall': { 'input': 'value', 'dec': 0.01, 'min': 0, 'max': 10, 'typical': 0.53},			
		'u_value_door': { 'input': 'value', 'dec': 0.01, 'min': 0, 'max': 10, 'typical': 4.65},			
		'u_value_window': { 'input': 'value', 'dec': 0.01, 'min': 0, 'max': 10, 'typical': 4.65},			
		'u_value_floor_bathroom': { 'input': 'value', 'dec': 0.01, 'min': 0, 'max': 10, 'typical': 0.48, 'conditions': [{'insulation_type_bathroom': 1}]},			
		'u_value_floor_other': { 'input': 'value', 'dec': 0.01, 'min': 0, 'max': 10, 'typical': 0.48, 'conditions': [{'insulation_type': 1}]},			
		'is_psi_value_base_input': { 'input': 'option', 'options': [1, 2], 'typical': 1},			
		'psi_value_earthfloor_perimeter_entrance': { 'input': 'value', 'dec': 0.01, 'min': 0, 'max': 10, 'typical': 0.53, 'conditions': [{'is_psi_value_base_input': 1}]},			
		'psi_value_earthfloor_perimeter_bathroom': { 'input': 'value', 'dec': 0.01, 'min': 0, 'max': 10, 'typical': 0.53, 'conditions': [{'is_psi_value_base_input': 1, 'insulation_type_bathroom': 2}]},			
		'psi_value_earthfloor_perimeter_other': { 'input': 'value', 'dec': 0.01, 'min': 0, 'max': 10, 'typical': 0.53, 'conditions': [{'is_psi_value_base_input': 1, 'insulation_type': 2}]},			
		'eta_d_value_window_h': { 'input': 'value', 'dec': 0.001, 'min': 0, 'max': 1, 'typical': 0.63},			
		'eta_d_value_window_c': { 'input': 'value', 'dec': 0.001, 'min': 0, 'max': 1, 'typical': 0.63},			
		'is_f_value_input': { 'input': 'option', 'options': [1, 2], 'typical': 1},			
		'f_value_h': { 'input': 'value', 'dec': 0.001, 'min': 0, 'max': 1, 'typical': 0.589, 'conditions': [{'is_f_value_input': 1}]},			
		'f_value_c': { 'input': 'value', 'dec': 0.001, 'min': 0, 'max': 1, 'typical': 0.864, 'conditions': [{'is_f_value_input': 1}]}			
	}},				
	'general_parts': {'conditions': [{'input_method': 3}, {'input_method': 4}], 'n_list': 3, 'list': [{				
		'name': { 'input': 'text', 'templete': 'GP'},			
		'general_part_type': { 'input': 'option', 'options': [1, 2, 3, 4, 5, 6, 7], 'typical': 1},			
		'next_space': { 'input': 'option', 'options': [1, 2, 3, 4, 5, 6], 'typical': 1},			
		'direction': { 'input': 'option', 'options': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13], 'typical': 1},			
		'area': { 'input': 'value', 'dec': 0.01, 'min': 0, 'max': 99.99, 'typical': 1},			
		'space_type': { 'input': 'option', 'options': [1, 2, 3, 4], 'typical': 1, 'conditions': [{'input_method': 4}]},			
		'spec': {'dictionary': {			
			'structure': { 'input': 'option', 'options': [1, 2, 3, 4], 'typical': 1},		
			'u_value_input_method_wood': { 'input': 'option', 'options': [1, 2, 3, '(4'], 'exception': ['(4'], 'typical': 1, 'conditions': [{'structure': 1}]},		
			'u_value_input_method_rc': { 'input': 'option', 'options': [1, 2], 'typical': 1, 'conditions': [{'structure': 2}]},		
			'u_value_input_method_steel': { 'input': 'option', 'options': [1, 2], 'typical': 1, 'conditions': [{'structure': 3}]},		
			'u_value_wood': { 'input': 'value', 'dec': 0.01, 'min': 0, 'max': 10, 'typical': 1, 'conditions': [{'u_value_input_method_wood': 1}]},		
			'floor_construction_method': { 'input': 'option', 'options': [1, 2, 3, 4, 5, 6, 7], 'typical': 1, 'conditions': [{'general_part_type': 4, 'u_value_input_method_wood': 3}, {'general_part_type': 6, 'u_value_input_method_wood': 3}, {'general_part_type': 7, 'u_value_input_method_wood': 3}]},		
			'wall_construction_method': { 'input': 'option', 'options': [1, 2, 3, 4, 5, 6], 'typical': 1, 'conditions': [{'general_part_type': 3, 'u_value_input_method_wood': 3}, {'general_part_type': 5, 'u_value_input_method_wood': 3}]},		
			'ceiling_construction_method': { 'input': 'option', 'options': [1], 'typical': 1, 'conditions': [{'general_part_type': 2, 'u_value_input_method_wood': 3}]},		
			'roof_construction_method': { 'input': 'option', 'options': [1, 2], 'typical': 1, 'conditions': [{'general_part_type': 1, 'u_value_input_method_wood': 3}]},		
			'u_value_rc': { 'input': 'value', 'dec': 0.01, 'min': 0, 'max': 10, 'typical': 1, 'conditions': [{'u_value_input_method_rc': 1}]},		
			'u_value_steel': { 'input': 'value', 'dec': 0.01, 'min': 0, 'max': 10, 'typical': 1, 'conditions': [{'u_value_input_method_steel': 1}]},		
			'u_r_value_steel': { 'input': 'value', 'dec': 0.01, 'min': 0, 'max': 0.7, 'typical': 1, 'conditions': [{'u_value_input_method_steel': 2}]},		
			'u_value_other': { 'input': 'value', 'dec': 0.01, 'min': 0, 'max': 10, 'typical': 1, 'conditions': [{'structure': 4}]},		
			'parts': {'conditions': [{'u_value_input_method_wood': 2}, {'u_value_input_method_wood': 3}, {'u_value_input_method_rc': 2}, {'u_value_input_method_steel': 2}], 'n_list': {1: [{'u_value_input_method_wood': 2}, {'u_value_input_method_rc': 2}, {'u_value_input_method_steel': 2}], 2: [{'u_value_input_method_wood': 3}], 4: [{'general_part_type': 1, 'u_value_input_method_wood': 3}, {'general_part_type': 3, 'u_value_input_method_wood': 3}, {'general_part_type': 4, 'u_value_input_method_wood': 3}, {'general_part_type': 5, 'u_value_input_method_wood': 3}], 6: [{'general_part_type': 3, 'u_value_input_method_wood': 3}, {'general_part_type': 5, 'u_value_input_method_wood': 3}]}, 'list': [{		
				'name': { 'input': 'text', 'templete': 'PRT'},	
				'part_type': { 'input': 'option', 'options': [1, 2, 3, 4, 5, 6, 7, 8], 'typical': 1, 'conditions': [{'u_value_input_method_wood': 3}]},	
				'layers': {'n_list': 3, 'list': [{	
					'name': { 'input': 'text', 'templete': 'LYR'},
					'heat_resistance_input_method': { 'input': 'option', 'options': [1, 2], 'typical': 1},
					'thermal_conductivity': { 'input': 'value', 'dec': 0.001, 'min': 0.001, 'max': 999.999, 'typical': 1, 'conditions': [{'heat_resistance_input_method': 1}]},
					'thermal_resistance': { 'input': 'value', 'dec': 0.1, 'min': 0, 'max': 9999.9, 'typical': 1, 'conditions': [{'heat_resistance_input_method': 2}]},
					'thickness': { 'input': 'value', 'dec': 0.0001, 'min': 0.0001, 'max': 9.9999, 'typical': 1},
					'volumetric_specific_heat': { 'input': 'value', 'dec': 0.1, 'min': 0, 'max': 9999.9, 'typical': 1}
				}]}	
			}]},		
			'is_sunshade_input': { 'input': 'option', 'options': [1, 2], 'typical': 1}		
		}}			
	}]},				
	'windows': {'n_list': 3, 'list': [{				
		'name': { 'input': 'text', 'templete': 'WND'},			
		'next_space': { 'input': 'option', 'options': [1, 2, 3, 4, 5, 6], 'typical': 1},			
		'direction': { 'input': 'option', 'options': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13], 'typical': 1},			
		'area': { 'input': 'value', 'dec': 0.01, 'min': 0, 'max': 99.99, 'typical': 1},			
		'space_type': { 'input': 'option', 'options': [1, 2, 3, 4], 'typical': 1},			
		'spec': {'dictionary': {			
			'window_type': { 'input': 'option', 'options': [1, 2], 'typical': 1},		
			'windows': {'n_list': {1: [{'window_type': 1}], 2: [{'window_type': 2}]}, 'list': [{		
				'u_value_input_method': { 'input': 'option', 'options': [1, 2], 'typical': 1},	
				'u_vaue': { 'input': 'value', 'dec': 0.01, 'min': 0, 'max': 10, 'typical': 1, 'conditions': [{'u_value_input_method': 1}]},	
				'ug_value': { 'input': 'value', 'dec': 0.01, 'min': 0, 'max': 10, 'typical': 1, 'conditions': [{'u_value_input_method': 2}]},	
				'flame_type': { 'input': 'option', 'options': [1, 2, 3], 'typical': 1, 'conditions': [{'u_value_input_method': 2}]},	
				'eta_d_value': { 'input': 'value', 'dec': 0.001, 'min': 0, 'max': 1, 'typical': 1},	
				'glass_type': { 'input': 'option', 'options': [1, 2, 3, 4], 'typical': 1}	
			}]},		
			'attachment_type': { 'input': 'option', 'options': [1, 2, 3, 4], 'typical': 1},		
			'is_windbreak_room_attached': { 'input': 'option', 'options': [1, 2], 'typical': 1},		
			'sunshade': {'dictionary': {		
				'y1': { 'input': 'value', 'dec': 0.01, 'min': 0, 'max': 10, 'typical': 1},	
				'y2': { 'input': 'value', 'dec': 0.01, 'min': 0, 'max': 10, 'typical': 1},	
				'z': { 'input': 'value', 'dec': 0.01, 'min': 0, 'max': 10, 'typical': 1}	
			}}		
		}}			
	}]},				
	'doors': {'n_list': 3, 'list': [{				
		'name': { 'input': 'text', 'templete': 'DR'},			
		'next_space': { 'input': 'option', 'options': [1, 2, 3, 4, 5, 6], 'typical': 1},			
		'direction': { 'input': 'option', 'options': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13], 'typical': 1},			
		'area': { 'input': 'value', 'dec': 0.01, 'min': 0, 'max': 99.99, 'typical': 1},			
		'space_type': { 'input': 'option', 'options': [1, 2, 3, 4], 'typical': 1},			
		'spec': {'dictionary': {			
			'u_vaue': { 'input': 'value', 'dec': 0.01, 'min': 0, 'max': 10, 'typical': 1},		
			'is_sunshade_input': { 'input': 'option', 'options': [1, 2], 'typical': 1}		
		}}			
	}]},				
	'earthfloor_perimeters': {'n_list': 3, 'list': [{				
		'name': { 'input': 'text', 'templete': 'EFP'},			
		'next_space': { 'input': 'option', 'options': [1, 2, 3, 4, 5, 6], 'typical': 1},			
		'direction': { 'input': 'option', 'options': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13], 'typical': 1},			
		'length': { 'input': 'value', 'dec': 0.01, 'min': 0, 'max': 99.99, 'typical': 1},			
		'space_type': { 'input': 'option', 'options': [1, 2, 3, 4], 'typical': 1},			
		'spec': {'dictionary': {			
			'psi_value': { 'input': 'value', 'dec': 0.01, 'min': 0, 'max': 10, 'typical': 1}		
		}}			
	}]},				
	'earthfloor_centers': {'n_list': 3, 'list': [{				
		'name': { 'input': 'text', 'templete': 'EFC'},			
		'area': { 'input': 'value', 'dec': 0.01, 'min': 0, 'max': 99.99, 'typical': 1},			
		'space_type': { 'input': 'option', 'options': [1, 2, 3, 4], 'typical': 1}			
	}]},				
	'inner_floors': {'n_list': 3, 'list': [{				
		'name': { 'input': 'text', 'templete': 'INF'},			
		'area': { 'input': 'value', 'dec': 0.01, 'min': 0, 'max': 99.99, 'typical': 1},			
		'upper_space_type': { 'input': 'option', 'options': [1, 2, 3, 4], 'typical': 1},			
		'lower_space_type': { 'input': 'option', 'options': [1, 2, 3, 4], 'typical': 1},			
		'spec': {'dictionary': {			
			'layers': {'n_list': 3, 'list': [{		
				'name': { 'input': 'text', 'templete': 'LYR'},	
				'heat_resistance_input_method': { 'input': 'option', 'options': [1, 2], 'typical': 1},	
				'thermal_conductivity': { 'input': 'value', 'dec': 0.001, 'min': 0.001, 'max': 999.999, 'typical': 1, 'conditions': [{'heat_resistance_input_method': 1}]},	
				'thermal_resistance': { 'input': 'value', 'dec': 0.1, 'min': 0, 'max': 9999.9, 'typical': 1, 'conditions': [{'heat_resistance_input_method': 2}]},	
				'thickness': { 'input': 'value', 'dec': 0.0001, 'min': 0.0001, 'max': 9.9999, 'typical': 1},	
				'volumetric_specific_heat': { 'input': 'value', 'dec': 0.1, 'min': 0, 'max': 9999.9, 'typical': 1}	
			}]}		
		}}			
	}]},				
	'inner_walls': {'n_list': 3, 'list': [{				
		'name': { 'input': 'text', 'templete': 'INW'},			
		'area': { 'input': 'value', 'dec': 0.01, 'min': 0, 'max': 99.99, 'typical': 1},			
		'space_type_1': { 'input': 'option', 'options': [1, 2, 3, 4], 'typical': 1},			
		'speace_type_2': { 'input': 'option', 'options': [1, 2, 3, 4], 'typical': 1},			
		'spec': {'dictionary': {			
			'layers': {'n_list': 3, 'list': [{		
				'name': { 'input': 'text', 'templete': 'LYR'},	
				'heat_resistance_input_method': { 'input': 'option', 'options': [1, 2], 'typical': 1},	
				'thermal_conductivity': { 'input': 'value', 'dec': 0.001, 'min': 0.001, 'max': 999.999, 'typical': 1, 'conditions': [{'heat_resistance_input_method': 1}]},	
				'thermal_resistance': { 'input': 'value', 'dec': 0.1, 'min': 0, 'max': 9999.9, 'typical': 1, 'conditions': [{'heat_resistance_input_method': 2}]},	
				'thickness': { 'input': 'value', 'dec': 0.0001, 'min': 0.0001, 'max': 9.9999, 'typical': 1},	
				'volumetric_specific_heat': { 'input': 'value', 'dec': 0.1, 'min': 0, 'max': 9999.9, 'typical': 1}	
			}]}		
		}}			
	}]}				
}},					
'ventilation': {'dictionary': {					
	'air_change_rate': { 'input': 'value', 'dec': 0.1, 'min': 0, 'max': 1, 'typical': 1}				
}}					
}

output_cases(keys_and_settings)